In [1]:
import os
import glob
import pickle
from datetime import datetime
import time
import dotenv
import pandas as pd

import requests
import requests.auth

import praw

from typing import List

import openai

import langchain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

from pydantic import BaseModel, Field, validator

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# load secrets from .env into environment variables
dotenv.load_dotenv()

langchain.__version__

'0.0.154'

- interrogate praw docs and write code to download from reddit
- do some language stuff and extract the songs
- interrogate spotipy write code to look up the uri
- interrogate spotipy, write code to get the playlist id
- interrogate spotipy, write code to update the playlist



In [ ]:
# models_obj = openai.Model.list()
# sorted([m['id'] for m in models_obj['data']])

# add pinecone
# do the csv thing

In [ ]:
# this doesn't work for some reason, end up in version hell
# loader = DirectoryLoader(hftcdir, glob="*.txt")
# d=loader.load_and_split()


In [13]:
hftcdir = '/Users/drucev/notebooks/llamaindex/gpt_index/examples/hftc'

documents = None
for f in glob.glob('%s/*' % hftcdir):

#     print(f)
    if documents is None:
        documents = TextLoader(f).load()
    else:
        documents.extend(TextLoader(f).load())
    

In [4]:
embedding_model = 'text-embedding-ada-002'   # default

# split the documents into chunks
print(1, 'splitting', datetime.now())
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
# load embeddings into vector db
print(2, 'getting embeddings', datetime.now())
embeddings = OpenAIEmbeddings(model=embedding_model,
                              deployment = embedding_model,
)
# create the vectorestore to use as the index
db = Chroma.from_documents(texts, embeddings)
# expose this index in a retriever interface
print(4, 'instantiate retriever', datetime.now())
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})


1 splitting 2023-05-02 13:38:44.493895


Using embedded DuckDB without persistence: data will be transient


2 getting embeddings 2023-05-02 13:38:45.904285
4 instantiate retriever 2023-05-02 13:43:43.606870


In [5]:
qa_model = 'gpt-3.5-turbo-0301'

llm = ChatOpenAI(model_name=qa_model, 
                 temperature=0.2,
                )

qa = RetrievalQA.from_chain_type(llm=llm, 
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True
                                )


In [ ]:
# create a chain to answer questions
# print(5, datetime.now())
# qa = RetrievalQA.from_chain_type(llm=OpenAI(model_name=qa_model, 
#                                             temperature=0.5), 
#                                  chain_type="stuff", 
#                                  retriever=retriever, 
#                                  return_source_documents=True)


In [21]:
query = "what are some leading endpoint protection products?"
result = qa({"query": query})
print(result['result'])

The email doesn't provide a comprehensive list of leading endpoint protection products, but it does mention two: Sophos (the current vendor) and Symantec (used at the author's previous job). Other popular endpoint protection products include McAfee, Kaspersky, Trend Micro, and Bitdefender. It's important to do research and evaluate different products to determine which one best fits your specific needs.


In [18]:
query = "what are the names of some good Managed Service Providers?"
result = qa({"query": query})
print(result['result'])


The email mentions three Managed Service Providers: AlphaServe, CDI, and Agio. However, it does not provide any opinions or recommendations about them.


In [19]:
query = "what are the names of leading Manged Security Service Providers or MDRs or MSSPs?"
result = qa({"query": query})
result['result']


"There are several leading Managed Security Service Providers (MSSPs) and Managed Detection and Response (MDR) companies in the market. Some of the top MSSPs include IBM Security, SecureWorks, AT&T Cybersecurity, Verizon, and Trustwave. Some of the top MDR companies include CrowdStrike, FireEye, Cybereason, Carbon Black, and Palo Alto Networks. However, it's important to note that the best provider for your company will depend on your specific needs and requirements."